# VacationPy

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
import gmaps
from scipy import stats
import pandas as pd
from config import gkey

In [2]:
# Read CSV generated by WeatherPy

weather_df = pd.read_csv("../WeatherPy/data/weather_data.csv")
weather_df.head(10)

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date Time
0,Belmonte,BR,-15.86,-38.88,80.60,61.0,40.0,9.17,1601056674
1,Soledade,BR,-28.82,-52.51,80.60,42.0,20.0,11.41,1601056729
2,Tuktoyaktuk,CA,69.45,-133.04,35.60,100.0,90.0,14.99,1601056437
3,Wanning,CN,18.80,110.40,79.59,80.0,99.0,1.97,1601056386
4,Khatanga,RU,71.97,102.50,38.97,85.0,99.0,8.52,1601056437
5,Sur,OM,22.57,59.53,82.60,63.0,0.0,8.70,1601056435
6,Khatassy,RU,61.90,129.64,46.40,49.0,92.0,4.47,1601056534
7,Kampong Thum,KH,12.71,104.89,76.64,84.0,94.0,2.89,1601056534
8,Hobart,AU,-42.88,147.33,46.40,87.0,90.0,4.70,1601056318
9,Najran,SA,17.49,44.13,86.00,16.0,0.0,6.93,1601056431


## Heat Map of Humidity

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Get Humidity
humidity = weather_df["Humidity (%)"].astype(float)

In [5]:
# Create heatmat of humidity with above data on world map with Google maps
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

![Heatmap of Humidity](images/heatmap.png)

In [6]:
# Tweak heatmap layer
heat_layer.point_radius = 10
heat_layer.dissipating=True

## Reduce dataset to ideal weather

In [7]:
# Create new data frame of cities with a max temperature between 72-77 degrees, wind speed less than 10 mph,
# and up to 1% cloudiness.

ideal_weather = weather_df.loc[((weather_df["Temperature (F)"] >= 72) & (weather_df["Temperature (F)"] <= 77) & 
                                (weather_df["Cloudiness (%)"] <= 1) & (weather_df["Wind Speed (mph)"] < 10)),:]
# Check how many cities found
ideal_weather.count()

City                10
Country             10
Latitude            10
Longitude           10
Temperature (F)     10
Humidity (%)        10
Cloudiness (%)      10
Wind Speed (mph)    10
Date Time           10
dtype: int64

In [8]:
# Display cities with ideal weather data
ideal_weather

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date Time
15,Benton Harbor,US,42.12,-86.45,77.00,44.0,1.0,7.07,1601056521
18,Cheyenne,US,41.14,-104.82,72.00,18.0,1.0,4.70,1601056700
70,Easton,US,40.69,-75.22,75.99,56.0,1.0,4.70,1601056750
82,Witu,KE,-2.39,40.44,75.83,81.0,1.0,9.98,1601056754
235,Viransehir,TR,37.24,39.76,74.91,39.0,0.0,5.68,1601056815
279,Zhob,PK,31.34,69.45,73.42,18.0,0.0,3.04,1601056836
329,Nanakuli,US,21.39,-158.15,77.00,78.0,1.0,3.36,1601056860
411,Bardiyah,LY,31.76,25.09,75.36,74.0,0.0,8.97,1601056893
421,Loreto,MX,22.27,-101.97,74.28,25.0,0.0,4.00,1601056899
438,Makakilo City,US,21.35,-158.09,77.00,78.0,1.0,3.36,1601056903


In [9]:
# Create new data frame from ideal weather with just the info we want for our hotel data
hotel_data = ideal_weather.loc[:,["City", "Country", "Latitude", "Longitude"]]
hotel_data

,City,Country,Latitude,Longitude
15,Benton Harbor,US,42.12,-86.45
18,Cheyenne,US,41.14,-104.82
70,Easton,US,40.69,-75.22
82,Witu,KE,-2.39,40.44
235,Viransehir,TR,37.24,39.76
279,Zhob,PK,31.34,69.45
329,Nanakuli,US,21.39,-158.15
411,Bardiyah,LY,31.76,25.09
421,Loreto,MX,22.27,-101.97
438,Makakilo City,US,21.35,-158.09


## Get data on hotels in top 10 cities according to ideal weather

In [10]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_data.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    long = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{long}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and convert to json
    get_hotel = requests.get(base_url, params=params).json()
    
    #print(json.dumps(get_hotel, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        # Add to column for hotel name
        hotel_data.loc[index, "Hotel Name"] = get_hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
# Print hotel list
hotel_data

Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Latitude,Longitude,Hotel Name
15,Benton Harbor,US,42.12,-86.45,The Inn At Harbor Shores
18,Cheyenne,US,41.14,-104.82,Tru by Hilton Cheyenne
70,Easton,US,40.69,-75.22,Hampton Inn Easton
82,Witu,KE,-2.39,40.44,NaN
235,Viransehir,TR,37.24,39.76,Tella Hotel
279,Zhob,PK,31.34,69.45,Rohail hotel Zhob
329,Nanakuli,US,21.39,-158.15,NaN
411,Bardiyah,LY,31.76,25.09,Hotel Burdy
421,Loreto,MX,22.27,-101.97,Hotel San Miguel
438,Makakilo City,US,21.35,-158.09,Marriott's Ko Olina Beach Club


In [11]:
# Drop rows with no hotel name
hotel_destinations = hotel_data.dropna(how="any")
hotel_destinations

,City,Country,Latitude,Longitude,Hotel Name
15,Benton Harbor,US,42.12,-86.45,The Inn At Harbor Shores
18,Cheyenne,US,41.14,-104.82,Tru by Hilton Cheyenne
70,Easton,US,40.69,-75.22,Hampton Inn Easton
235,Viransehir,TR,37.24,39.76,Tella Hotel
279,Zhob,PK,31.34,69.45,Rohail hotel Zhob
411,Bardiyah,LY,31.76,25.09,Hotel Burdy
421,Loreto,MX,22.27,-101.97,Hotel San Miguel
438,Makakilo City,US,21.35,-158.09,Marriott's Ko Olina Beach Club


In [12]:
# Save hotel data to CSV
hotel_destinations.to_csv("data/hotels.csv", index=False)

In [13]:
# Read hotel data from CSV
hotel_destinations = pd.read_csv("data/hotels.csv")
hotel_destinations

,City,Country,Latitude,Longitude,Hotel Name
0,Benton Harbor,US,42.12,-86.45,The Inn At Harbor Shores
1,Cheyenne,US,41.14,-104.82,Tru by Hilton Cheyenne
2,Easton,US,40.69,-75.22,Hampton Inn Easton
3,Viransehir,TR,37.24,39.76,Tella Hotel
4,Zhob,PK,31.34,69.45,Rohail hotel Zhob
5,Bardiyah,LY,31.76,25.09,Hotel Burdy
6,Loreto,MX,22.27,-101.97,Hotel San Miguel
7,Makakilo City,US,21.35,-158.09,Marriott's Ko Olina Beach Club


## Add hotels to heat map

In [14]:
# Get coordinates
coordinates = hotel_destinations[["Latitude","Longitude"]].values.tolist()
coordinates


[[42.12, -86.45],
 [41.14, -104.82],
 [40.69, -75.22],
 [37.24, 39.76],
 [31.34, 69.45],
 [31.76, 25.09],
 [22.27, -101.97],
 [21.35, -158.09]]

In [15]:
# Create list for hover text
info = []

# Loop through data frame
for index, row in hotel_destinations.iterrows():
    # Generate hover text, use HTML to display nicely
    info.append(f'<b>City:</b><br>{row["City"]}<br><br><b>Country:</b><br>{row["Country"]}<br><br><b>Hotel:</b><br>{row["Hotel Name"]}')


In [16]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates,info_box_content=info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

![Hotels in Ideal Weather Cities](images/hotel_pins.png)